## Library Installation

Ensure that the necessary libraries are installed before running the notebook.

In [18]:
%pip install openpyxl
from openpyxl import load_workbook
import pandas as pd
import os
import re
import itertools

print("Skeleton setup complete!")


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Skeleton setup complete!


## Variable Declaration

Set the variables for file paths, sheet names, and other configurations. Update these variables for each specific project.

In [19]:
# Path to the Excel file (change this for each project)
excel_file_path = '/workspaces/Finetwork-Automation/inbound/Metabase.xlsx'
csv_file_path = '/workspaces/Finetwork-Automation/inbound/Informe de métricas históricas.csv'

# Sheet names for different categories
sheet_active = 'ACTIVOS'
sheet_canceled = 'CANCELADOS'
sheet_pending = 'PTE DE FIRMA'

# Range to read (change this for each project)
start_row = 8
end_row = 65
usecols = 'A:AF'

print("Variables defined correctly!")

Variables defined correctly!


## Verify Columns in 'Active' Sheet.

Verify the number of columns in the "Active" sheet to ensure the range is within bounds. We will use the range in all three sheets

In [20]:
# Function to verify the number of columns
def verify_columns(file_path, sheet_name):
    workbook = load_workbook(filename=file_path, data_only=True)
    sheet = workbook[sheet_name]
    max_column = sheet.max_column
    return max_column

# Check the number of columns in the 'Active' sheet
max_column_active = verify_columns(excel_file_path, 'ACTIVOS')
print(f"Max column in 'Active' sheet: {max_column_active}")

# Check if the number of columns matches the expected range
expected_columns = 32  # Columns from A to AF (inclusive)
if max_column_active < expected_columns:
    usecols = f"A:{chr(64+max_column_active)}"
    print(f"Adjusted usecols to: {usecols}")
else:
    print(f"Using default usecols: {usecols}")

Max column in 'Active' sheet: 14
Adjusted usecols to: A:N


## Extract Data from all three Sheets

Extract data from all three Sheets within the specified range and convert it directly to a DataFrame.

In [21]:
def load_sheet_as_dataframe(file_path, sheet_name, start_row, end_row, usecols):
    # Load data from the specified sheet and range into a DataFrame
    df = pd.read_excel(file_path, sheet_name=sheet_name, usecols=usecols, skiprows=start_row-1, nrows=end_row-start_row+1)
    print(f"Data from '{sheet_name}' sheet loaded successfully.")
    return df

# Extract data from 'Active' sheet
active_df = load_sheet_as_dataframe(excel_file_path, 'ACTIVOS', start_row, end_row, usecols)
canceled_df = load_sheet_as_dataframe(excel_file_path, 'CANCELADOS', start_row, end_row, usecols)
pending_signature_df = load_sheet_as_dataframe(excel_file_path, 'PTE FIRMA', start_row, end_row, usecols)

# Display the DataFrames
print("Active DataFrame:")
display(active_df.head())

print("Canceled DataFrame:")
display(canceled_df.head())

print("Pending Signature DataFrame:")
display(pending_signature_df.head())

Data from 'ACTIVOS' sheet loaded successfully.
Data from 'CANCELADOS' sheet loaded successfully.
Data from 'PTE FIRMA' sheet loaded successfully.
Active DataFrame:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,2024-08-09 00:00:00,2024-08-10 00:00:00,2024-08-11 00:00:00,2024-08-12 00:00:00,Total general
0,Inbound Telec.Orig.Sevilla,328.0,281.0,168.0,109.0,364.0,372.0,384.0,333.0,382.0,184.0,118.0,474.0,3497
1,david.molero@originaltelecom.es,9.0,5.0,NaN,NaN,17.0,16.0,26.0,20.0,29.0,NaN,NaN,32.0,154
2,miguel.segura@originaltelecom.es,17.0,17.0,NaN,NaN,13.0,17.0,19.0,8.0,18.0,NaN,NaN,34.0,143
3,francisco.perdomo@originaltelecom.es,17.0,13.0,NaN,NaN,18.0,14.0,21.0,16.0,21.0,NaN,NaN,14.0,134
4,gonzalofalcon@originaltelecom.es,14.0,12.0,NaN,NaN,12.0,15.0,21.0,22.0,15.0,NaN,NaN,21.0,132


Canceled DataFrame:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,2024-08-09 00:00:00,2024-08-10 00:00:00,2024-08-11 00:00:00,2024-08-12 00:00:00,Total general
0,Inbound Telec.Orig.Sevilla,150.0,142.0,78.0,60.0,159.0,143.0,125.0,104.0,63.0,26.0,16.0,61.0,1127
1,francisco.perdomo@originaltelecom.es,11.0,8.0,NaN,NaN,7.0,8.0,2.0,3.0,3.0,NaN,NaN,10.0,52
2,laura.eguens@originaltelecom.es,6.0,9.0,NaN,NaN,7.0,2.0,10.0,8.0,8.0,NaN,NaN,1.0,51
3,miriam.rodriguez@originaltelecom.es,6.0,6.0,NaN,NaN,3.0,8.0,10.0,12.0,1.0,NaN,NaN,4.0,50
4,david.molero@originaltelecom.es,7.0,5.0,NaN,NaN,10.0,12.0,1.0,11.0,NaN,NaN,NaN,2.0,48


Pending Signature DataFrame:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,2024-08-09 00:00:00,2024-08-10 00:00:00,2024-08-11 00:00:00,2024-08-12 00:00:00,Total general
0,Inbound Telec.Orig.Sevilla,13.0,3.0,NaN,NaN,1.0,6.0,14.0,15.0,13.0,10.0,9.0,63.0,147
1,david.molero@originaltelecom.es,NaN,NaN,NaN,NaN,NaN,NaN,1.0,4.0,1.0,NaN,NaN,5.0,11
2,maricarmen.cornejo@originaltelecom.es,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,2.0,NaN,6.0,9
3,anasanchez@originaltelecom.es,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,2.0,8
4,antonio.reina@originaltelecom.es,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,1.0,NaN,NaN,4.0,8


## Replace NaN with 0

Replace all NaN values in the DataFrames with 0 to facilitate further transformations.

In [22]:
def replace_nan_with_zero(df):
    """
    Replace all NaN values in the DataFrame with 0.
    
    Parameters:
    df (pd.DataFrame): The DataFrame to process.
    
    Returns:
    pd.DataFrame: The processed DataFrame with NaN replaced by 0.
    """
    df = df.fillna(0)
    print("Replaced NaN with 0.")
    return df

active_df = replace_nan_with_zero(active_df)
canceled_df = replace_nan_with_zero(canceled_df)
pending_signature_df = replace_nan_with_zero(pending_signature_df)

# Display the transformed DataFrames
print("Active DataFrame after replacing NaN:")
display(active_df.head())

print("Canceled DataFrame after replacing NaN:")
display(canceled_df.head())

print("Pending Signature DataFrame after replacing NaN:")
display(pending_signature_df.head())

Replaced NaN with 0.
Replaced NaN with 0.
Replaced NaN with 0.
Active DataFrame after replacing NaN:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,2024-08-09 00:00:00,2024-08-10 00:00:00,2024-08-11 00:00:00,2024-08-12 00:00:00,Total general
0,Inbound Telec.Orig.Sevilla,328.0,281.0,168.0,109.0,364.0,372.0,384.0,333.0,382.0,184.0,118.0,474.0,3497
1,david.molero@originaltelecom.es,9.0,5.0,0.0,0.0,17.0,16.0,26.0,20.0,29.0,0.0,0.0,32.0,154
2,miguel.segura@originaltelecom.es,17.0,17.0,0.0,0.0,13.0,17.0,19.0,8.0,18.0,0.0,0.0,34.0,143
3,francisco.perdomo@originaltelecom.es,17.0,13.0,0.0,0.0,18.0,14.0,21.0,16.0,21.0,0.0,0.0,14.0,134
4,gonzalofalcon@originaltelecom.es,14.0,12.0,0.0,0.0,12.0,15.0,21.0,22.0,15.0,0.0,0.0,21.0,132


Canceled DataFrame after replacing NaN:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,2024-08-09 00:00:00,2024-08-10 00:00:00,2024-08-11 00:00:00,2024-08-12 00:00:00,Total general
0,Inbound Telec.Orig.Sevilla,150.0,142.0,78.0,60.0,159.0,143.0,125.0,104.0,63.0,26.0,16.0,61.0,1127
1,francisco.perdomo@originaltelecom.es,11.0,8.0,0.0,0.0,7.0,8.0,2.0,3.0,3.0,0.0,0.0,10.0,52
2,laura.eguens@originaltelecom.es,6.0,9.0,0.0,0.0,7.0,2.0,10.0,8.0,8.0,0.0,0.0,1.0,51
3,miriam.rodriguez@originaltelecom.es,6.0,6.0,0.0,0.0,3.0,8.0,10.0,12.0,1.0,0.0,0.0,4.0,50
4,david.molero@originaltelecom.es,7.0,5.0,0.0,0.0,10.0,12.0,1.0,11.0,0.0,0.0,0.0,2.0,48


Pending Signature DataFrame after replacing NaN:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,2024-08-09 00:00:00,2024-08-10 00:00:00,2024-08-11 00:00:00,2024-08-12 00:00:00,Total general
0,Inbound Telec.Orig.Sevilla,13.0,3.0,0.0,0.0,1.0,6.0,14.0,15.0,13.0,10.0,9.0,63.0,147
1,david.molero@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,1.0,0.0,0.0,5.0,11
2,maricarmen.cornejo@originaltelecom.es,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,6.0,9
3,anasanchez@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,2.0,8
4,antonio.reina@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,4.0,8


## Load Agents List

Load the list of all agents from the "Agents" sheet.

In [23]:
# Load the list of agents
agents_df = pd.read_excel(excel_file_path, sheet_name='AGENTES', usecols='A')
agents_list = agents_df.iloc[:, 0].tolist()
print("Agents list loaded successfully!")
print(agents_list)

Agents list loaded successfully!
['albaaraujo@originaltelecom.es', 'albertocanto@originaltelecom.es', 'albertosanchez@originaltelecom.es', 'anasanchez@originaltelecom.es', 'antonio.reina@originaltelecom.es', 'azahara.garcia@originaltelecom.es', 'beatriz.gomez@originaltelecom.es', 'maricarmen.cornejo@originaltelecom.es', 'carolinafuentes@originaltelecom.es', 'cesar.arnaldo@originaltelecom.es', 'david.molero@originaltelecom.es', 'elenaborrero@originaltelecom.es', 'estefania.panea@originaltelecom.es', 'francisco.perdomo@originaltelecom.es', 'gonzalofalcon@originaltelecom.es', 'guillermo.hurtado@originaltelecom.es', 'irati.izaguirre@originaltelecom.es', 'ivan.barroso@originaltelecom.es', 'laura.eguens@originaltelecom.es', 'lailasetati@originaltelecom.es', 'leonor.lopez@originaltelecom.es', 'dolores.cortes@originaltelecom.es', 'manuelvaldes@originaltelecom.es', 'manuelventura@originaltelecom.es', 'mar.aguila@originaltelecom.es', 'mariangeles.bueso@originaltelecom.es', 'mariaarroyo@originalt

## Verify and Complete Data

Verify that all agents are present in each DataFrame. If an agent is missing, add a row with zeros for that agent.

In [24]:
def ensure_all_agents(df, agents_list):
    """
    Ensure all agents are present in the DataFrame. Add missing agents with zero values and remove agents not in the list.
    
    Parameters:
    df (pd.DataFrame): The DataFrame to check and update.
    agents_list (list): The list of all agents.
    
    Returns:
    pd.DataFrame: The updated DataFrame with all agents.
    """
    # Get the list of agents in the DataFrame
    existing_agents = df.iloc[:, 0].tolist()
    
    # Find missing agents
    missing_agents = [agent for agent in agents_list if agent not in existing_agents]
    
    # Add rows for missing agents with zero values
    for agent in missing_agents:
        zero_row = pd.DataFrame([[agent] + [0] * (df.shape[1] - 1)], columns=df.columns)
        df = pd.concat([df, zero_row], ignore_index=True)
    
    # Remove agents not in the agents list
    df = df[df.iloc[:, 0].isin(agents_list)]
    
    print(f"Added {len(missing_agents)} missing agents and removed {len(existing_agents) - len(df)} agents not in the list.")
    return df

# Apply the function to each DataFrame
active_df = ensure_all_agents(active_df, agents_list)
canceled_df = ensure_all_agents(canceled_df, agents_list)
pending_signature_df = ensure_all_agents(pending_signature_df, agents_list)

# Display the updated DataFrames
print("Active DataFrame after ensuring all agents:")
display(active_df.head(8))

print("Canceled DataFrame after ensuring all agents:")
display(canceled_df.head(8))

print("Pending Signature DataFrame after ensuring all agents:")
display(pending_signature_df.head(88))

Added 2 missing agents and removed 15 agents not in the list.
Added 2 missing agents and removed 15 agents not in the list.
Added 9 missing agents and removed 2 agents not in the list.
Active DataFrame after ensuring all agents:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,2024-08-09 00:00:00,2024-08-10 00:00:00,2024-08-11 00:00:00,2024-08-12 00:00:00,Total general
1,david.molero@originaltelecom.es,9.0,5.0,0.0,0.0,17.0,16.0,26.0,20.0,29.0,0.0,0.0,32.0,154
2,miguel.segura@originaltelecom.es,17.0,17.0,0.0,0.0,13.0,17.0,19.0,8.0,18.0,0.0,0.0,34.0,143
3,francisco.perdomo@originaltelecom.es,17.0,13.0,0.0,0.0,18.0,14.0,21.0,16.0,21.0,0.0,0.0,14.0,134
4,gonzalofalcon@originaltelecom.es,14.0,12.0,0.0,0.0,12.0,15.0,21.0,22.0,15.0,0.0,0.0,21.0,132
5,albertosanchez@originaltelecom.es,15.0,10.0,0.0,0.0,10.0,15.0,33.0,21.0,11.0,0.0,0.0,13.0,128
7,leonor.lopez@originaltelecom.es,16.0,9.0,0.0,0.0,16.0,12.0,14.0,14.0,15.0,0.0,0.0,18.0,114
8,guillermo.hurtado@originaltelecom.es,6.0,13.0,17.0,7.0,1.0,0.0,10.0,10.0,11.0,18.0,18.0,0.0,111
9,laura.eguens@originaltelecom.es,8.0,7.0,0.0,0.0,9.0,17.0,7.0,21.0,26.0,0.0,0.0,15.0,110


Canceled DataFrame after ensuring all agents:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,2024-08-09 00:00:00,2024-08-10 00:00:00,2024-08-11 00:00:00,2024-08-12 00:00:00,Total general
1,francisco.perdomo@originaltelecom.es,11.0,8.0,0.0,0.0,7.0,8.0,2.0,3.0,3.0,0.0,0.0,10.0,52
2,laura.eguens@originaltelecom.es,6.0,9.0,0.0,0.0,7.0,2.0,10.0,8.0,8.0,0.0,0.0,1.0,51
3,miriam.rodriguez@originaltelecom.es,6.0,6.0,0.0,0.0,3.0,8.0,10.0,12.0,1.0,0.0,0.0,4.0,50
4,david.molero@originaltelecom.es,7.0,5.0,0.0,0.0,10.0,12.0,1.0,11.0,0.0,0.0,0.0,2.0,48
5,guillermo.hurtado@originaltelecom.es,7.0,12.0,7.0,1.0,0.0,0.0,9.0,2.0,0.0,6.0,3.0,0.0,47
6,anasanchez@originaltelecom.es,0.0,0.0,13.0,8.0,12.0,2.0,4.0,0.0,0.0,0.0,7.0,0.0,46
7,sergio.vazquez@originaltelecom.es,3.0,8.0,0.0,0.0,3.0,1.0,3.0,10.0,4.0,0.0,0.0,10.0,42
8,elenaborrero@originaltelecom.es,3.0,5.0,0.0,0.0,4.0,7.0,0.0,6.0,15.0,0.0,0.0,0.0,40


Pending Signature DataFrame after ensuring all agents:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,2024-08-09 00:00:00,2024-08-10 00:00:00,2024-08-11 00:00:00,2024-08-12 00:00:00,Total general
1,david.molero@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,1.0,0.0,0.0,5.0,11
2,maricarmen.cornejo@originaltelecom.es,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,6.0,9
3,anasanchez@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,2.0,8
4,antonio.reina@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,4.0,8
5,patricia.rios@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,0.0,0.0,1.0,7
6,albertosanchez@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,2.0,6
7,pilar.deval@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,5.0,6
8,albertocanto@originaltelecom.es,4.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,6
10,laura.eguens@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5
11,miguel.segura@originaltelecom.es,0.0,2.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,5


## Assign Values to Emails

Assign numerical values to each email and add them as a new column in the DataFrames.

In [25]:
# Dictionary mapping emails to their respective values
email_values = {
    'albaaraujo@originaltelecom.es': 1,
    'albertocanto@originaltelecom.es': 2,
    'albertosanchez@originaltelecom.es': 3,
    'anasanchez@originaltelecom.es': 4,
    'antonio.reina@originaltelecom.es': 5,
    'azahara.garcia@originaltelecom.es': 6,
    'beatriz.gomez@originaltelecom.es': 7,
    'maricarmen.cornejo@originaltelecom.es': 8,
    'carolinafuentes@originaltelecom.es': 9,
    'cesar.arnaldo@originaltelecom.es': 10,
    'david.molero@originaltelecom.es': 11,
    'elenaborrero@originaltelecom.es': 12,
    'estefania.panea@originaltelecom.es': 13,
    'francisco.perdomo@originaltelecom.es': 14,
    'gonzalofalcon@originaltelecom.es': 15,
    'guillermo.hurtado@originaltelecom.es': 16,
    'irati.izaguirre@originaltelecom.es': 17,
    'ivan.barroso@originaltelecom.es': 18,
    'laura.eguens@originaltelecom.es': 19,
    'lailasetati@originaltelecom.es': 20,
    'leonor.lopez@originaltelecom.es': 21,
    'dolores.cortes@originaltelecom.es': 22,
    'manuelvaldes@originaltelecom.es': 23,
    'manuelventura@originaltelecom.es': 24,
    'mar.aguila@originaltelecom.es': 25,
    'mariangeles.bueso@originaltelecom.es': 26,
    'mariaarroyo@originaltelecom.es': 27,
    'maria.torres@originaltelecom.es': 28,
    'marta.dorado@originaltelecom.es': 29,
    'mauricio.pozo@originaltelecom.es': 30,
    'miguel.segura@originaltelecom.es': 31,
    'miriam.rodriguez@originaltelecom.es': 32,
    'mar.marchena@originaltelecom.es': 33,
    'natividad.sanchez@originaltelecom.es': 34,
    'nereacerezo@originaltelecom.es': 35,
    'oscar.rivilla@originaltelecom.es': 36,
    'patricia.rios@originaltelecom.es': 37,
    'paulavilla@originaltelecom.es': 38,
    'pilar.deval@originaltelecom.es': 39,
    'sara.elkhelyfy@originaltelecom.es': 40,
    'sergio.vazquez@originaltelecom.es': 41,
    'yicel.patricia@originaltelecom.es': 42,
    'yzabelly.gomes@originaltelecom.es': 43
}

# Add a new column to each DataFrame with the email values
def add_email_values(df, email_values):
    df['email_value'] = df.iloc[:, 0].map(email_values)
    return df

# Apply the function to each DataFrame
active_df = add_email_values(active_df, email_values)
canceled_df = add_email_values(canceled_df, email_values)
pending_signature_df = add_email_values(pending_signature_df, email_values)

# Display the updated DataFrames with the new 'email_value' column
print("Active DataFrame with email values:")
display(active_df.head(65))

print("Canceled DataFrame with email values:")
display(canceled_df.head(65))

print("Pending Signature DataFrame with email values:")
display(pending_signature_df.head(65))

Active DataFrame with email values:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,2024-08-09 00:00:00,2024-08-10 00:00:00,2024-08-11 00:00:00,2024-08-12 00:00:00,Total general,email_value
1,david.molero@originaltelecom.es,9.0,5.0,0.0,0.0,17.0,16.0,26.0,20.0,29.0,0.0,0.0,32.0,154,11
2,miguel.segura@originaltelecom.es,17.0,17.0,0.0,0.0,13.0,17.0,19.0,8.0,18.0,0.0,0.0,34.0,143,31
3,francisco.perdomo@originaltelecom.es,17.0,13.0,0.0,0.0,18.0,14.0,21.0,16.0,21.0,0.0,0.0,14.0,134,14
4,gonzalofalcon@originaltelecom.es,14.0,12.0,0.0,0.0,12.0,15.0,21.0,22.0,15.0,0.0,0.0,21.0,132,15
5,albertosanchez@originaltelecom.es,15.0,10.0,0.0,0.0,10.0,15.0,33.0,21.0,11.0,0.0,0.0,13.0,128,3
7,leonor.lopez@originaltelecom.es,16.0,9.0,0.0,0.0,16.0,12.0,14.0,14.0,15.0,0.0,0.0,18.0,114,21
8,guillermo.hurtado@originaltelecom.es,6.0,13.0,17.0,7.0,1.0,0.0,10.0,10.0,11.0,18.0,18.0,0.0,111,16
9,laura.eguens@originaltelecom.es,8.0,7.0,0.0,0.0,9.0,17.0,7.0,21.0,26.0,0.0,0.0,15.0,110,19
10,mariangeles.bueso@originaltelecom.es,0.0,0.0,21.0,14.0,24.0,18.0,0.0,0.0,0.0,17.0,13.0,0.0,107,26
11,carolinafuentes@originaltelecom.es,8.0,8.0,0.0,0.0,12.0,18.0,11.0,19.0,17.0,0.0,0.0,14.0,107,9


Canceled DataFrame with email values:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,2024-08-09 00:00:00,2024-08-10 00:00:00,2024-08-11 00:00:00,2024-08-12 00:00:00,Total general,email_value
1,francisco.perdomo@originaltelecom.es,11.0,8.0,0.0,0.0,7.0,8.0,2.0,3.0,3.0,0.0,0.0,10.0,52,14
2,laura.eguens@originaltelecom.es,6.0,9.0,0.0,0.0,7.0,2.0,10.0,8.0,8.0,0.0,0.0,1.0,51,19
3,miriam.rodriguez@originaltelecom.es,6.0,6.0,0.0,0.0,3.0,8.0,10.0,12.0,1.0,0.0,0.0,4.0,50,32
4,david.molero@originaltelecom.es,7.0,5.0,0.0,0.0,10.0,12.0,1.0,11.0,0.0,0.0,0.0,2.0,48,11
5,guillermo.hurtado@originaltelecom.es,7.0,12.0,7.0,1.0,0.0,0.0,9.0,2.0,0.0,6.0,3.0,0.0,47,16
6,anasanchez@originaltelecom.es,0.0,0.0,13.0,8.0,12.0,2.0,4.0,0.0,0.0,0.0,7.0,0.0,46,4
7,sergio.vazquez@originaltelecom.es,3.0,8.0,0.0,0.0,3.0,1.0,3.0,10.0,4.0,0.0,0.0,10.0,42,41
8,elenaborrero@originaltelecom.es,3.0,5.0,0.0,0.0,4.0,7.0,0.0,6.0,15.0,0.0,0.0,0.0,40,12
9,mariaarroyo@originaltelecom.es,4.0,8.0,0.0,0.0,8.0,9.0,2.0,3.0,6.0,0.0,0.0,0.0,40,27
10,antonio.reina@originaltelecom.es,1.0,12.0,0.0,0.0,9.0,10.0,3.0,0.0,4.0,0.0,0.0,1.0,40,5


Pending Signature DataFrame with email values:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,2024-08-09 00:00:00,2024-08-10 00:00:00,2024-08-11 00:00:00,2024-08-12 00:00:00,Total general,email_value
1,david.molero@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,1.0,0.0,0.0,5.0,11,11
2,maricarmen.cornejo@originaltelecom.es,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,6.0,9,8
3,anasanchez@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,2.0,8,4
4,antonio.reina@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,4.0,8,5
5,patricia.rios@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,0.0,0.0,1.0,7,37
6,albertosanchez@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,2.0,6,3
7,pilar.deval@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,5.0,6,39
8,albertocanto@originaltelecom.es,4.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,6,2
10,laura.eguens@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5,19
11,miguel.segura@originaltelecom.es,0.0,2.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,5,31


## Sort DataFrames by Email Values

Sort the DataFrames based on the numerical values assigned to the emails.

In [26]:
# Sort each DataFrame by the 'email_value' column
def sort_by_email_value(df):
    df = df.sort_values(by='email_value')
    return df

# Apply the sorting function to each DataFrame
active_df = sort_by_email_value(active_df)
canceled_df = sort_by_email_value(canceled_df)
pending_signature_df = sort_by_email_value(pending_signature_df)

# Display the sorted DataFrames
print("Sorted Active DataFrame:")
display(active_df.head(88))  # Displaying first 20 rows for testing

print("Sorted Canceled DataFrame:")
display(canceled_df.head(88))  # Displaying first 20 rows for testing

print("Sorted Pending Signature DataFrame:")
display(pending_signature_df.head(88))  # Displaying first 20 rows for testing

Sorted Active DataFrame:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,2024-08-09 00:00:00,2024-08-10 00:00:00,2024-08-11 00:00:00,2024-08-12 00:00:00,Total general,email_value
19,albaaraujo@originaltelecom.es,10.0,9.0,0.0,0.0,8.0,22.0,7.0,6.0,21.0,0.0,0.0,9.0,92,1
17,albertocanto@originaltelecom.es,8.0,7.0,0.0,0.0,13.0,8.0,14.0,12.0,17.0,0.0,0.0,15.0,94,2
5,albertosanchez@originaltelecom.es,15.0,10.0,0.0,0.0,10.0,15.0,33.0,21.0,11.0,0.0,0.0,13.0,128,3
12,anasanchez@originaltelecom.es,0.0,0.0,13.0,11.0,16.0,10.0,16.0,0.0,0.0,18.0,8.0,13.0,105,4
21,antonio.reina@originaltelecom.es,10.0,5.0,0.0,0.0,5.0,6.0,11.0,18.0,15.0,0.0,0.0,14.0,84,5
39,azahara.garcia@originaltelecom.es,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,7.0,10.0,0.0,29,6
22,beatriz.gomez@originaltelecom.es,6.0,9.0,0.0,0.0,7.0,6.0,10.0,9.0,18.0,0.0,0.0,16.0,81,7
23,maricarmen.cornejo@originaltelecom.es,0.0,0.0,0.0,0.0,9.0,10.0,0.0,0.0,0.0,27.0,21.0,12.0,79,8
11,carolinafuentes@originaltelecom.es,8.0,8.0,0.0,0.0,12.0,18.0,11.0,19.0,17.0,0.0,0.0,14.0,107,9
28,cesar.arnaldo@originaltelecom.es,0.0,0.0,16.0,7.0,5.0,0.0,0.0,0.0,0.0,20.0,4.0,10.0,62,10


Sorted Canceled DataFrame:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,2024-08-09 00:00:00,2024-08-10 00:00:00,2024-08-11 00:00:00,2024-08-12 00:00:00,Total general,email_value
32,albaaraujo@originaltelecom.es,1.0,0.0,0.0,0.0,3.0,5.0,3.0,2.0,0.0,0.0,0.0,1.0,15,1
21,albertocanto@originaltelecom.es,1.0,7.0,0.0,0.0,0.0,5.0,5.0,3.0,0.0,0.0,0.0,3.0,24,2
15,albertosanchez@originaltelecom.es,4.0,1.0,0.0,0.0,5.0,4.0,5.0,2.0,4.0,0.0,0.0,3.0,28,3
6,anasanchez@originaltelecom.es,0.0,0.0,13.0,8.0,12.0,2.0,4.0,0.0,0.0,0.0,7.0,0.0,46,4
10,antonio.reina@originaltelecom.es,1.0,12.0,0.0,0.0,9.0,10.0,3.0,0.0,4.0,0.0,0.0,1.0,40,5
37,azahara.garcia@originaltelecom.es,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,12,6
18,beatriz.gomez@originaltelecom.es,6.0,6.0,0.0,0.0,3.0,3.0,4.0,3.0,1.0,0.0,0.0,0.0,26,7
30,maricarmen.cornejo@originaltelecom.es,0.0,0.0,0.0,0.0,1.0,4.0,0.0,0.0,0.0,6.0,0.0,4.0,15,8
16,carolinafuentes@originaltelecom.es,2.0,1.0,0.0,0.0,5.0,5.0,3.0,5.0,3.0,0.0,0.0,2.0,26,9
26,cesar.arnaldo@originaltelecom.es,0.0,0.0,10.0,5.0,2.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,21,10


Sorted Pending Signature DataFrame:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,2024-08-09 00:00:00,2024-08-10 00:00:00,2024-08-11 00:00:00,2024-08-12 00:00:00,Total general,email_value
12,albaaraujo@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5,1
8,albertocanto@originaltelecom.es,4.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,6,2
6,albertosanchez@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,2.0,6,3
3,anasanchez@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,2.0,8,4
4,antonio.reina@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,4.0,8,5
25,azahara.garcia@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2,6
14,beatriz.gomez@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4,7
2,maricarmen.cornejo@originaltelecom.es,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,6.0,9,8
20,carolinafuentes@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,3,9
37,cesar.arnaldo@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,10


## Remove 'email_value' Column

After sorting the DataFrames based on the email values, the 'email_value' column should be removed to prevent interference with further calculations.

In [27]:
# Function to remove the 'email_value' column
def remove_email_value_column(df):
    if 'email_value' in df.columns:
        df = df.drop(columns=['email_value'])
    return df

# Apply the function to each DataFrame
active_df = remove_email_value_column(active_df)
canceled_df = remove_email_value_column(canceled_df)
pending_signature_df = remove_email_value_column(pending_signature_df)

# Display the updated DataFrames without the 'email_value' column
print("Active DataFrame after removing 'email_value' column:")
display(active_df.head(88))

print("Canceled DataFrame after removing 'email_value' column:")
display(canceled_df.head(88))

print("Pending Signature DataFrame after removing 'email_value' column:")
display(pending_signature_df.head(88))

column_values = active_df.iloc[:, 0].tolist()
print("Values from the second column of active_df:")
print(column_values)

Active DataFrame after removing 'email_value' column:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,2024-08-09 00:00:00,2024-08-10 00:00:00,2024-08-11 00:00:00,2024-08-12 00:00:00,Total general
19,albaaraujo@originaltelecom.es,10.0,9.0,0.0,0.0,8.0,22.0,7.0,6.0,21.0,0.0,0.0,9.0,92
17,albertocanto@originaltelecom.es,8.0,7.0,0.0,0.0,13.0,8.0,14.0,12.0,17.0,0.0,0.0,15.0,94
5,albertosanchez@originaltelecom.es,15.0,10.0,0.0,0.0,10.0,15.0,33.0,21.0,11.0,0.0,0.0,13.0,128
12,anasanchez@originaltelecom.es,0.0,0.0,13.0,11.0,16.0,10.0,16.0,0.0,0.0,18.0,8.0,13.0,105
21,antonio.reina@originaltelecom.es,10.0,5.0,0.0,0.0,5.0,6.0,11.0,18.0,15.0,0.0,0.0,14.0,84
39,azahara.garcia@originaltelecom.es,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,7.0,10.0,0.0,29
22,beatriz.gomez@originaltelecom.es,6.0,9.0,0.0,0.0,7.0,6.0,10.0,9.0,18.0,0.0,0.0,16.0,81
23,maricarmen.cornejo@originaltelecom.es,0.0,0.0,0.0,0.0,9.0,10.0,0.0,0.0,0.0,27.0,21.0,12.0,79
11,carolinafuentes@originaltelecom.es,8.0,8.0,0.0,0.0,12.0,18.0,11.0,19.0,17.0,0.0,0.0,14.0,107
28,cesar.arnaldo@originaltelecom.es,0.0,0.0,16.0,7.0,5.0,0.0,0.0,0.0,0.0,20.0,4.0,10.0,62


Canceled DataFrame after removing 'email_value' column:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,2024-08-09 00:00:00,2024-08-10 00:00:00,2024-08-11 00:00:00,2024-08-12 00:00:00,Total general
32,albaaraujo@originaltelecom.es,1.0,0.0,0.0,0.0,3.0,5.0,3.0,2.0,0.0,0.0,0.0,1.0,15
21,albertocanto@originaltelecom.es,1.0,7.0,0.0,0.0,0.0,5.0,5.0,3.0,0.0,0.0,0.0,3.0,24
15,albertosanchez@originaltelecom.es,4.0,1.0,0.0,0.0,5.0,4.0,5.0,2.0,4.0,0.0,0.0,3.0,28
6,anasanchez@originaltelecom.es,0.0,0.0,13.0,8.0,12.0,2.0,4.0,0.0,0.0,0.0,7.0,0.0,46
10,antonio.reina@originaltelecom.es,1.0,12.0,0.0,0.0,9.0,10.0,3.0,0.0,4.0,0.0,0.0,1.0,40
37,azahara.garcia@originaltelecom.es,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,12
18,beatriz.gomez@originaltelecom.es,6.0,6.0,0.0,0.0,3.0,3.0,4.0,3.0,1.0,0.0,0.0,0.0,26
30,maricarmen.cornejo@originaltelecom.es,0.0,0.0,0.0,0.0,1.0,4.0,0.0,0.0,0.0,6.0,0.0,4.0,15
16,carolinafuentes@originaltelecom.es,2.0,1.0,0.0,0.0,5.0,5.0,3.0,5.0,3.0,0.0,0.0,2.0,26
26,cesar.arnaldo@originaltelecom.es,0.0,0.0,10.0,5.0,2.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,21


Pending Signature DataFrame after removing 'email_value' column:


,Etiquetas de fila,2024-08-01 00:00:00,2024-08-02 00:00:00,2024-08-03 00:00:00,2024-08-04 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,2024-08-09 00:00:00,2024-08-10 00:00:00,2024-08-11 00:00:00,2024-08-12 00:00:00,Total general
12,albaaraujo@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5
8,albertocanto@originaltelecom.es,4.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,6
6,albertosanchez@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,2.0,6
3,anasanchez@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,2.0,8
4,antonio.reina@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,4.0,8
25,azahara.garcia@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2
14,beatriz.gomez@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4
2,maricarmen.cornejo@originaltelecom.es,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,6.0,9
20,carolinafuentes@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,3
37,cesar.arnaldo@originaltelecom.es,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1


Values from the second column of active_df:
['albaaraujo@originaltelecom.es', 'albertocanto@originaltelecom.es', 'albertosanchez@originaltelecom.es', 'anasanchez@originaltelecom.es', 'antonio.reina@originaltelecom.es', 'azahara.garcia@originaltelecom.es', 'beatriz.gomez@originaltelecom.es', 'maricarmen.cornejo@originaltelecom.es', 'carolinafuentes@originaltelecom.es', 'cesar.arnaldo@originaltelecom.es', 'david.molero@originaltelecom.es', 'elenaborrero@originaltelecom.es', 'estefania.panea@originaltelecom.es', 'francisco.perdomo@originaltelecom.es', 'gonzalofalcon@originaltelecom.es', 'guillermo.hurtado@originaltelecom.es', 'irati.izaguirre@originaltelecom.es', 'ivan.barroso@originaltelecom.es', 'laura.eguens@originaltelecom.es', 'lailasetati@originaltelecom.es', 'leonor.lopez@originaltelecom.es', 'dolores.cortes@originaltelecom.es', 'manuelvaldes@originaltelecom.es', 'manuelventura@originaltelecom.es', 'mar.aguila@originaltelecom.es', 'mariangeles.bueso@originaltelecom.es', 'mariaarroy

## Define Useful Columns for all three DataFrames

Determine the range of columns that will be used for all three DataFrames.

In [28]:
# Determine the range of columns for Active DataFrame
active_usecols = active_df.columns[:-1]
canceled_usecols = canceled_df.columns[:-1]
pending_signature_usecols = pending_signature_df.columns[:-1]


print("Active DataFrame Useful Columns:")
print(active_usecols)

print("Canceled DataFrame Useful Columns:")
print(canceled_usecols)

print("Pending Signature DataFrame Useful Columns:")
print(pending_signature_usecols)

Active DataFrame Useful Columns:
Index(['Etiquetas de fila', 2024-08-01 00:00:00, 2024-08-02 00:00:00,
       2024-08-03 00:00:00, 2024-08-04 00:00:00, 2024-08-05 00:00:00,
       2024-08-06 00:00:00, 2024-08-07 00:00:00, 2024-08-08 00:00:00,
       2024-08-09 00:00:00, 2024-08-10 00:00:00, 2024-08-11 00:00:00,
       2024-08-12 00:00:00],
      dtype='object')
Canceled DataFrame Useful Columns:
Index(['Etiquetas de fila', 2024-08-01 00:00:00, 2024-08-02 00:00:00,
       2024-08-03 00:00:00, 2024-08-04 00:00:00, 2024-08-05 00:00:00,
       2024-08-06 00:00:00, 2024-08-07 00:00:00, 2024-08-08 00:00:00,
       2024-08-09 00:00:00, 2024-08-10 00:00:00, 2024-08-11 00:00:00,
       2024-08-12 00:00:00],
      dtype='object')
Pending Signature DataFrame Useful Columns:
Index(['Etiquetas de fila', 2024-08-01 00:00:00, 2024-08-02 00:00:00,
       2024-08-03 00:00:00, 2024-08-04 00:00:00, 2024-08-05 00:00:00,
       2024-08-06 00:00:00, 2024-08-07 00:00:00, 2024-08-08 00:00:00,
       2024-08-0

## Update Diario Agosto Sheet with Active DataFrame

For each useful column in the Active DataFrame, update the corresponding column in the "DIARIO AGOSTO" sheet.

In [29]:
final_file_path ='/workspaces/Finetwork-Automation/inbound/OBJETIVO_CALL_INB_AGOSTO_24.xlsx'
sheet_name = 'GLOBAL AGOSTO'

def update_diario_agosto_with_last_column(df, file_path):
    """
    Update the "DIARIO AGOSTO" sheet with values from the last column of the Active DataFrame.
    
    Parameters:
    df (pd.DataFrame): The Active DataFrame.
    file_path (str): Path to the Excel file.
    """
    workbook = load_workbook(filename=file_path)
    sheet = workbook['DIARIO AGOSTO']
    
    # Identify the last and penultimate columns in the DataFrame
    last_column_name = df.columns[-2]
    penultimate_column_index = len(df.columns) - 2  # Get the index of the penultimate column
    
    # Define the column mappings: Active DataFrame -> DIARIO AGOSTO
    column_mappings = {
        2: 'C',   # Assuming 'B' is the first column
        3: 'H',   # Assuming 'C' is the second column
        4: 'M',   # Assuming 'D' is the third column
        5: 'R',   # Assuming 'E' is the fourth column
        6: 'AB',  # Assuming 'F' is the fifth column
        7: 'AG',
        8: 'AL',
        9: 'AQ',
        10: 'AV',
        11: 'BA',
        12: 'BF',
        13: 'BK',
        14: 'BP',
        15: 'BU',
        16: 'BZ',
        17: 'CE',
        18: 'CJ',
        19: 'CO',
        20: 'CT',
        21: 'CY',
        22: 'DD',
        23: 'DI',
        24: 'DN',
        25: 'DS',
        26: 'DX',
        27: 'EC',
        28: 'EH',
        29: 'EM',
        30: 'ER',
        31: 'EW'
    }
    
    # Determine the target column based on the penultimate column index
    target_column = column_mappings.get(penultimate_column_index, 'EW')  # Default to 'EW' if index not found
    
    # Update the Excel sheet with values from the last column of the DataFrame
    for row_idx, value in enumerate(df[last_column_name], start=4):  # Assuming Excel updates start at row 4
        sheet[f'{target_column}{row_idx}'] = value
    
    workbook.save(file_path)
    print(f"Updated DIARIO AGOSTO sheet with the last column '{last_column_name}' from Active DataFrame.")

# Update the Diario Agosto sheet with the last column of the Active DataFrame
update_diario_agosto_with_last_column(active_df, final_file_path)

Updated DIARIO AGOSTO sheet with the last column '2024-08-12 00:00:00' from Active DataFrame.


## Update finalFile Excel Sheet with Canceled Data

Update the "finalFile" Excel sheet with the calculated values from the Canceled DataFrame in column N.

In [30]:
def update_diario_agosto_with_last_column_canceled(df, file_path):
    """
    Update the "DIARIO AGOSTO" sheet with values from the last column of the Canceled DataFrame.
    
    Parameters:
    df (pd.DataFrame): The Canceled DataFrame.
    file_path (str): Path to the Excel file.
    """
    workbook = load_workbook(filename=file_path)
    sheet = workbook['DIARIO AGOSTO']
    
    # Identify the last and penultimate columns in the DataFrame
    last_column_name = df.columns[-2]
    penultimate_column_index = len(df.columns) - 2  # Get the index of the penultimate column
    
    # Define the column mappings: Canceled DataFrame -> DIARIO AGOSTO
    column_mappings = {
        2: 'D',   
        3: 'I',   
        4: 'N',   
        5: 'S',   
        6: 'AC',  
        7: 'AH',
        8: 'AM',
        9: 'AR',
        10: 'AW',
        11: 'BB',
        12: 'BG',
        13: 'BL',
        14: 'BQ',
        15: 'BV',
        16: 'CA',
        17: 'CF',
        18: 'CK',
        19: 'CP',
        20: 'CU',
        21: 'CZ',
        22: 'DE',
        23: 'DJ',
        24: 'DO',
        25: 'DT',
        26: 'DY',
        27: 'ED',
        28: 'EI',
        29: 'EN',
        30: 'ES',
        31: 'EX'
    }
    
    # Determine the target column based on the penultimate column index
    target_column = column_mappings.get(penultimate_column_index, 'EW')  # Default to 'EW' if index not found
    
    # Update the Excel sheet with values from the last column of the DataFrame
    for row_idx, value in enumerate(df[last_column_name], start=4):  # Assuming Excel updates start at row 4
        sheet[f'{target_column}{row_idx}'] = value
    
    workbook.save(file_path)
    print(f"Updated DIARIO AGOSTO sheet with the last column '{last_column_name}' from Canceled DataFrame.")

# Update the Diario Agosto sheet with the last column of the Canceled DataFrame
update_diario_agosto_with_last_column_canceled(canceled_df, final_file_path)

Updated DIARIO AGOSTO sheet with the last column '2024-08-12 00:00:00' from Canceled DataFrame.


## Update Diario Agosto Sheet with Pending Signature DataFrame

For each useful column in the Pending Signature DataFrame, update the corresponding column in the "DIARIO AGOSTO" sheet.

In [31]:
def update_diario_agosto_with_last_column_pending_signature(df, file_path):
    """
    Update the "DIARIO AGOSTO" sheet with values from the last column of the Pending Signature DataFrame.
    
    Parameters:
    df (pd.DataFrame): The Pending Signature DataFrame.
    file_path (str): Path to the Excel file.
    """
    workbook = load_workbook(filename=file_path)
    sheet = workbook['DIARIO AGOSTO']
    
    # Identify the last and penultimate columns in the DataFrame
    last_column_name = df.columns[-2]
    penultimate_column_index = len(df.columns) - 2  # Get the index of the penultimate column
    
    # Define the column mappings: Pending Signature DataFrame -> DIARIO AGOSTO
    column_mappings = {
        2: 'E',   # Adjusted to the next column
        3: 'J',   
        4: 'O',   
        5: 'T',   
        6: 'AD',  
        7: 'AI',
        8: 'AN',
        9: 'AS',
        10: 'AX',
        11: 'BC',
        12: 'BH',
        13: 'BM',
        14: 'BR',
        15: 'BW',
        16: 'CB',
        17: 'CG',
        18: 'CL',
        19: 'CQ',
        20: 'CV',
        21: 'DA',
        22: 'DF',
        23: 'DK',
        24: 'DP',
        25: 'DU',
        26: 'DZ',
        27: 'EE',
        28: 'EJ',
        29: 'EO',
        30: 'ET',
        31: 'EY'
    }
    
    # Determine the target column based on the penultimate column index
    target_column = column_mappings.get(penultimate_column_index, 'EY')  # Default to 'EY' if index not found
    
    # Update the Excel sheet with values from the last column of the DataFrame
    for row_idx, value in enumerate(df[last_column_name], start=4):  # Assuming Excel updates start at row 4
        sheet[f'{target_column}{row_idx}'] = value
    
    workbook.save(file_path)
    print(f"Updated DIARIO AGOSTO sheet with the last column '{last_column_name}' from Pending Signature DataFrame.")

# Update the Diario Agosto sheet with the last column of the Pending Signature DataFrame
update_diario_agosto_with_last_column_pending_signature(pending_signature_df, final_file_path)

Updated DIARIO AGOSTO sheet with the last column '2024-08-12 00:00:00' from Pending Signature DataFrame.
